In [29]:
import numpy as np
import sys

In [38]:
def lcg(a,b,y0,m,length):
    if m <= 0:
        raise ValueError("m must be a positive integer.")
    if ((a>=0 and a<m and (b>=0 and b<m) and (y0>=0 and y0<m))):
        y_current = y0
        result = []
        for i in range(0,length):
            y_next = (a*y_current + b)%m
            result.append(y_next)
            y_current = y_next
    else:
        raise ValueError("a, b, y0 must be integers between [0,m)")
    return result

In [54]:
pseudo_random_sequence = lcg(a=4,b=0,y0=2,m=252,length=10)
print(pseudo_random_sequence)

[8, 32, 128, 8, 32, 128, 8, 32, 128, 8]


In [55]:
# Calculates gcd of a pair of numbers
def gcd_euclidean_number_pair(a,b):
    if a == 0:
        return b;
    elif b == 0:
        return a;
    if a < 0:
        a = -a
    if b < 0:
        b = -b
    
    if a == b:
        return a;
    elif a > b:
        return gcd_euclidean_number_pair(a%b,b)
    elif a < b:
        return gcd_euclidean_number_pair (a,b%a)

# Calculates gcd of an array of numbers
def gcd_euclidean(inp):
    # take gcd of a pair and repeat the process
    gcd_num_pair = inp[0]
    for i in range(0,len(inp)-1):
        a = gcd_num_pair
        b = inp[i + 1]
        gcd_num_pair = gcd_euclidean_number_pair(a,b)
    gcd = gcd_num_pair
    return gcd

"""Implementation of the Extended Euclidean Algorithm to solve 
the Diophantine equation ax + by = 1 for integer values of x and y"""

"""
gcd(a,b) = gcd(b,a%b). Using this result the equation ax + by = gcd(a,b) can be modified into
b*x1 + (a%b)*y1 = gcd(a,b). Then a%b can be written as a - (a//b)*b. Substituting this into 
the second equation a recursive relation between (x1,x) and (y1,y) can be obtained. 

It is known that the last step of euclidean algorithm gives xn = 1 and yn = 0. 

xn-1 = y1
yn-1 = xn - (an-1//bn-1)*yn
"""
def extended_euclidean(a,b):
    
    if (gcd_euclidean([a,b]) != 1):
        raise ValueError("GCD of numbers not equal to 1. No integer solutions are possible")
    a_org = a
    b_org = b
    a_prev = a
    b_prev = b
    coeffs = [(a,b)]
    while (b!=0):
        a = b_prev
        b = a_prev%b_prev
        coeffs.append((a,b))
        a_prev = a
        b_prev = b  
    x = 1
    y = 0
    for i in range(0,len(coeffs)):
        a_prev = coeffs[len(coeffs) - i - 1][0]
        b_prev = coeffs[len(coeffs) - i - 1][1]
        if b_prev == 0:
            continue
        x_prev = y
        y_prev = x - (a_prev//b_prev)*y
        x = x_prev
        y = y_prev 
    # mod operation in python gives result with matching sign of denominator.
    if (a_org*x + b_org*y == -1):
        x = -x
        y = -y
    return x,y

In [58]:

def break_lcg(pseudo_random_sequence):
    sub_results = []
    for i in range(0,len(pseudo_random_sequence)-3):
        yn = pseudo_random_sequence[i]
        yn_1 = pseudo_random_sequence[i+1]
        yn_2 = pseudo_random_sequence[i+2]
        yn_3 = pseudo_random_sequence[i+3]
        """
        y2 - y3 = a(y1 - y2)(mod m) -(1)
        y3 - y4 = a(y2 - y3)(mod m) -(2)
        
        To eliminate 'a' multiply (1) with (y2 - y3) and (2) with (y1 - y2) and 
        subtract the equations
        
        """
        res = (yn_1 - yn_2)**2 - (yn_2 - yn_3)*(yn - yn_1)
        sub_results.append(res)
    
    m = gcd_euclidean(sub_results)
    y1 = pseudo_random_sequence[0]
    y2 = pseudo_random_sequence[1]
    y3 = pseudo_random_sequence[2]
    print(m)
    print(y1)
    print(y2)
    print(y3)
    """
    After finding m ,use eqn(1) to find a.
    
    """
    x,y = extended_euclidean(y1-y2,-m)
    a = ((y2 - y3)*x)%m
    
    """ After finding a and m, b can be found by y2 = a*y1 + b (mod m) """
    r = a*y1 - y2
    x,y = extended_euclidean(m,-1)
    b = (y*r)%m
    y1_b = y1 - b
    y0,x = extended_euclidean(a,-m)
    
    y0 = (y0 * y1_b)%m
    
    return a,b,m,y0

In [59]:
break_lcg(pseudo_random_sequence)

12096
8
32
128


ValueError: GCD of numbers not equal to 1. No integer solutions are possible